In [2]:
import sys

from sklearn.model_selection import train_test_split
from scipy.spatial.distance import cosine, correlation
from sklearn.metrics import pairwise_distances
from collections import defaultdict

In [18]:
import os
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine, correlation
from scipy.stats import norm
%matplotlib inline
import seaborn as sns
sns.set(color_codes=True)
from sklearn.cluster import KMeans



In [5]:
userFeature = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
ratingFeature = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
genreFeature = ['genre']
moviesFeature = ['movie_id', 'title', 'release_date', 'video_release_date','imdb_url']
train_file ='u1.base'
test_file = 'u1.test'

In [6]:
def importFile(fileName,feature,sep,crange):
    if sep == None:
        sep = '|'
    if crange == None:
        df = pd.read_csv(fileName, sep=sep,names=feature,encoding='latin-1')
    else:
        df = pd.read_csv(fileName, sep=sep,names=feature,usecols=range(crange),encoding='latin-1')
    return df

In [12]:
feature = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
df = pd.read_csv('~/Dev/datasets/ml-100k/u.data', 
                 sep='\t',
                 names=feature,
                 encoding='latin-1')

In [14]:
df.sample(10)

,user_id,movie_id,rating,unix_timestamp
5233,1,125,3,878542960
99128,927,38,5,879195783
84267,881,1177,1,876539418
81103,828,328,3,891033988
38595,454,454,3,881959745
64239,435,655,2,884131799
96907,911,473,3,892840996
3257,145,315,5,883840797
47796,160,276,5,876768106
93154,658,960,4,875147873


In [15]:
pivotRatingDf = df.pivot_table(index=['movie_id'],columns=['user_id'],values='rating').reset_index(drop=True)
pivotRatingDf.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
0,5.0,4.0,NaN,NaN,4.0,4.0,NaN,NaN,NaN,4.0,...,2.0,3.0,4.0,NaN,4.0,NaN,NaN,5.0,NaN,NaN
1,3.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,4.0,...,5.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN
4,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
pivotRatingDf.fillna(0,inplace=True)
pivotRatingDf.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
0,5.0,4.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,4.0,...,2.0,3.0,4.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0
1,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,...,5.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
4,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
X = pivotRatingDf[pivotRatingDf.columns[:]] # ไม่เอา 1 คอลัมแรก และ 2 คอลัมท้าย
X.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
0,5.0,4.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,4.0,...,2.0,3.0,4.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0
1,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,...,5.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
4,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
pivotRatingDf.to_csv('ratingMatrix.csv');

In [26]:
n_clusters = 3
kmeans = KMeans(n_clusters=n_clusters, n_init=10)
kmeans.fit(X)
labels = kmeans.labels_
print(labels)

[1 2 2 ... 0 0 0]


In [29]:
len(labels)

1682

In [31]:
pd.DataFrame(labels)

,0
0,1
1,2
2,2
3,1
4,2
5,0
6,1
7,1
8,1
9,2
